### Data preparation

#### Import packages

In [1]:
!pip install cvxopt


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# Install the needed libraries
import pandas as pd
import numpy as np
import cvxopt.modeling
from cvxopt.modeling import variable, op, matrix
import time
import warnings
from pandas.core.common import SettingWithCopyWarning

In [3]:
!pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [4]:
# Ignore SettingWithCopyWarning 
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

#### Import data

In [5]:
# Import the data
data = pd.read_excel('data_python_modified.xlsx', sheet_name='data', index_col=0)
real = pd.read_excel('data_python_modified.xlsx', sheet_name='реал', index_col=0)
bandwidth = pd.read_excel('data_python_modified.xlsx', sheet_name='ПС', index_col=0)
match = pd.read_excel('data_python_modified.xlsx', sheet_name='match', index_col=0)

#### Prepare data
* First we specify the functions that we use
* Then we apply them to get the final matrixes

Functions that we use for calculations:

#### Calculate the tarif

In [6]:
# Function for calculating the tarif
def calculate_tarif(row):
    if row["Плечо, км"] <= 50:
        return row["Тариф ж/д"] + row["Тариф хранение"] + row["Тариф бренд"] + row["Тариф ВЛ"]
    else:
        return row["Тариф ж/д"] + row["Тариф хранение"] + row["Тариф бренд"] +  row["Тариф ВЛ"] * row["Плечо, км"]

In [7]:
# Function for dividing supply between different brands
def divide_supply(brand_name, match_brand_product):
    
    product_name = match_brand_product[brand_name] 
    sum_brand = real.groupby('НГ')['Объем'].sum().loc[brand_name]
    sum_all = real.groupby('НГ_ПС')['Объем'].sum().loc[product_name]
    p_brand = sum_brand / sum_all
    
    return p_brand

Functions that we use for forming matrixes:

In [8]:
# Function for forming cost matrixes
def form_cost_matrixes(data):
    i = 0
    cost = {}
    for brand in data['НГ'].unique():
        string = "cost_brand_" + str(i)
        cost[string] = data[data['НГ']== brand].pivot_table(index=['НБ'], columns=['ОУ'], values='Тариф', dropna=False)
        i = i + 1
        
    return cost

In [9]:
# Function for forming demand matrixes
def form_demand_matrixes(data, real):
    i = 0
    demand = {}

    for brand in data['НГ'].unique():
        string = "demand_brand_" + str(i) +"_t"
        demand[string] = real[real['НГ']==brand][['ОУ','Объем']].groupby('ОУ').sum().transpose()
        i = i + 1
    
    return demand

In [10]:
# Function for forming supply matrixes
def form_supply_matrixes(data, bandwidth, match_brand_product):
    i = 0
    supply = {}

    for brand in data['НГ'].unique():
    
        product = match_brand_product[brand]
        percent_brand = divide_supply(brand, match_brand_product)
    
        string = "supply_brand_" + str(i)
        supply[string] = bandwidth[bandwidth['НГ_ПС'] == product].groupby('НБ').sum() * percent_brand
        i = i + 1
    
    return supply

In [11]:
# Function for forming final matrixes
def form_final_matrixes(cost, demand, supply):

    i = 0
    matrixes = {}

    for c, d, s in zip(cost.keys(), demand.keys(), supply.keys()):
        trans = pd.concat([cost[str(c)], demand[str(d)]], axis=0)
        string = "matrix_brand_" + str(i)
        matrixes[string] = pd.concat([trans, supply[str(s)]], axis=1)
        i = i + 1
        
    return matrixes

In [12]:
def prepare_data(data, real, bandwidth, match, t):
    # Calculate the tarif
    data['Тариф'] = data.apply(lambda row: calculate_tarif(row), axis=1)
    
    # Subset for the time period
    data = data[data['Дата'] == t]
    real = real[real['Дата'] == t]
    bandwidth = bandwidth[bandwidth['Дата'] == t]
    
    # Adjust datatypes
    data['НБ'] = data['НБ'].map(lambda x: int(x.strip('Нефтебаза ')))
    data['ОУ'] = data['ОУ'].map(lambda x: int(x.strip('АЗС ')))
    real['ОУ'] = real['ОУ'].map(lambda x: int(x.strip('АЗС ')))
    bandwidth['НБ'] = bandwidth['НБ'].map(lambda x: int(x.strip('Нефтебаза ')))
    
    # Replace np.inf with 100 000
    bandwidth.loc[:, "Объем"].replace(np.inf, 10**6, inplace=True)
    
    # Create some dictionaries
    match_brand_product = dict(zip(match['НГ'], match['НГ_ПС']))
    match_brand_number = dict(enumerate(match['НГ']))
    
    
    # Create transitionary matrixes
    cost = form_cost_matrixes(data)
    demand = form_demand_matrixes(data, real)
    supply = form_supply_matrixes(data, bandwidth, match_brand_product)
    
    # Create final matrixes
    matrixes = form_final_matrixes(cost, demand, supply)
    
    return matrixes, match_brand_number

Applying functions:

In [13]:
# Checking dates
dates = pd.DataFrame({"date" : data['Дата'].unique()})
dates

,date
0,2023-04-01
1,2023-03-01


In [14]:
# Save the first date to the t 
t = dates.iloc[0, 0]

In [15]:
# Get the matrixes
matrixes, match_brand_number = prepare_data(data, real, bandwidth, match, t)

In [16]:
# See example:
matrixes['matrix_brand_0']

,0,1,3,4,7,8,10,11,12,15,17,18,19,20,21,22,26,28,43,Объем
0,3268.488947,3241.752289,3232.346002,3245.001733,3205.039266,3331.254535,3217.866021,3237.077649,3238.046782,3285.249240,4259.389025,4439.413324,3159.376018,2935.734418,3000.837326,3228.469471,3200.421634,4756.964716,3264.612417,331.1
1,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,5402.787462,5595.115521,5424.934992,5210.404717,5272.855284,4808.465100,4808.465100,6504.271376,4808.465100,1000000.0
Объем,29.693803,23.717783,69.617399,17.180032,33.144700,19.220910,42.384663,18.717185,29.952396,8.060745,13.678923,8.212597,4.737933,15.236888,7.962316,14.279678,8.315718,10.245384,15.893195,NaN


In [17]:
match_brand_number

{0: 'Бензин 100 бренд',
 1: 'Бензин 92',
 2: 'Бензин 95',
 3: 'Бензин 95 бренд',
 4: 'Топливо дизельное с присадками летнее'}

### Preparation of data for cvxopt

In [18]:
def overall_data_lists(matrixes):
    result_costs = {}
    overall_costs = [] 
    result_demand = {}
    overall_demand = [] 
    values_n_points_brand = {}
    n_points_overall = 0
    result_supply = {}
    overall_supply = [] 
    
    for i, brand in enumerate(data['НГ'].unique()):
        matrix_brand = matrixes['matrix_brand_{i}'.format(i=i)]
        result_costs["values_costs_{}".format(i)] = np.array(matrix_brand[:-1].iloc[:,:-1].values).flatten().tolist()
        overall_costs += list(result_costs.values())[i]
        result_demand["values_demand_{}".format(i)] = pd.Series(matrix_brand[-1:].iloc[:,:-1].values.tolist())[0]
        overall_demand += list(result_demand.values())[i]
        values_n_points_brand["n_points_{}_t".format(i)] = len(matrix_brand.columns) - 1
        n_points_overall += values_n_points_brand['n_points_{i}_t'.format(i=i)]
        values_supply = matrixes['matrix_brand_{i}'.format(i=i)].iloc[:-1].loc[:, 'Объем']
        result_supply["values_supply_{}".format(i)] = values_supply.values.tolist()
        overall_supply += list(result_supply.values())[i]
        
    n_origins_sum = data['НБ'].nunique()
    n_origins_overall = len(overall_supply)
        
    return result_costs, overall_costs, result_demand, overall_demand, values_n_points_brand, n_points_overall, result_supply, overall_supply, n_origins_sum, n_origins_overall

In [19]:
result_costs, overall_costs, result_demand, overall_demand, values_n_points_brand, n_points_overall, result_supply, overall_supply, n_origins_sum, n_origins_overall = overall_data_lists(matrixes)

### Applying cvxopt to solve the problem

#### Generate a function we want to minimize

In [20]:
from cvxopt.modeling import variable, op, matrix

In [21]:
# Setting variables and costs for the model

x = variable(n_points_overall * n_origins_sum, 'x') #количество НБ, умноженное на количество АЗС. 180 * 2

c = matrix(np.array(overall_costs).reshape(len(x), 1)) # список костов из 360 ячеек. порядок:  100brand + 92 + 95 + 95brand + disel

In [22]:
z = cvxopt.modeling.dot(x, c)

In [23]:
print(z)

linear function of length 1
linear term: linear function of length 1
coefficient of variable(360,'x'):
[ 3.27e+03  3.24e+03  3.23e+03  3.25e+03  3.21e+03  3.33e+03  3.22e+03 ... ]



#### Generate restrictions on demand

In [24]:
number_of_constraints = 1
initial_n_of_constraints = number_of_constraints
number_of_x = 0
list_of_constraints = {}

for b in range(len(matrixes)):
    matrix_brand = matrixes['matrix_brand_{b}'.format(b=b)]
    for i in range(len(matrix_brand.columns)-1):
        constraint = 'mass' + str(number_of_constraints)
        
        trans = []
            
            
        string = ""
        
        for j in range(n_origins_sum):
            trans.append(x[i+number_of_x+j*(len(matrix_brand.columns)-1)])
            if j == n_origins_sum - 1:
                end = ''
            else:
                end = ' + '
            string += "x[{x}]{y}".format(x = i+number_of_x+j*(len(matrix_brand.columns)-1), y = end)
            
        list_of_constraints[constraint] = (cvxopt.modeling.sum(trans) == overall_demand[number_of_constraints - initial_n_of_constraints])
            
        print("mass{} = (".format(number_of_constraints) + string + " == " + str(overall_demand[number_of_constraints - initial_n_of_constraints]) + ")")
        print("")
        
        number_of_constraints += 1
        
    number_of_x += (len(matrix_brand.columns)-1) * n_origins_sum

mass1 = (x[0] + x[19] == 29.69380349634518)

mass2 = (x[1] + x[20] == 23.71778272032937)

mass3 = (x[2] + x[21] == 69.61739907583812)

mass4 = (x[3] + x[22] == 17.18003246835099)

mass5 = (x[4] + x[23] == 33.14470001081621)

mass6 = (x[5] + x[24] == 19.22090952214534)

mass7 = (x[6] + x[25] == 42.38466312255857)

mass8 = (x[7] + x[26] == 18.71718475620516)

mass9 = (x[8] + x[27] == 29.95239566535984)

mass10 = (x[9] + x[28] == 8.060744781665605)

mass11 = (x[10] + x[29] == 13.67892325874485)

mass12 = (x[11] + x[30] == 8.212596603459344)

mass13 = (x[12] + x[31] == 4.73793339941212)

mass14 = (x[13] + x[32] == 15.23688750856649)

mass15 = (x[14] + x[33] == 7.962316122394742)

mass16 = (x[15] + x[34] == 14.27967784390616)

mass17 = (x[16] + x[35] == 8.315718258401118)

mass18 = (x[17] + x[36] == 10.24538436128756)

mass19 = (x[18] + x[37] == 15.89319452782291)

mass20 = (x[38] + x[81] == 304.5709837527155)

mass21 = (x[39] + x[82] == 240.3026994876928)

mass22 = (x[40] + x[83] == 193.42

In [25]:
len(list_of_constraints)

180

#### Generate restrictions on supply

In [26]:
# Define restrictions on supply <= upper_const, supply >= lower_const    

# Percents of loading demanded. For now only try a first variable
percents_loading = np.arange(0.5, 1, 0.1)

number_of_x = 0
count = 0

for i, brand in enumerate(data['НГ'].unique()):
        supply_brand_i = matrixes['matrix_brand_{i}'.format(i=i)].iloc[:-1].loc[:, "Объем"]
        values_n_points_i = values_n_points_brand['n_points_{i}_t'.format(i=i)]
        
        for origin in range(n_origins_sum):
            
            constraint = 'mass' + str(number_of_constraints)
            trans = []
            
            N = number_of_x + values_n_points_i
            
            string = ""
        
            for j in range(number_of_x, N):
                trans.append(x[j])
                if j == N - 1:
                    end = ''
                else:
                    end = ' + '
                string += "x[{x}]{y}".format(x = j, y = end)
            
            list_of_constraints[constraint] = (cvxopt.modeling.sum(trans) <= overall_supply[count])
            
            print("mass{} = (".format(number_of_constraints) + string + ") <= " + str(overall_supply[count]) + ")")
            print("")
            number_of_constraints += 1
            constraint = 'mass' + str(number_of_constraints)
            const = np.min(percents_loading[0] * matrixes['matrix_brand_' + str(i)].iloc[:-1].loc[:, 'Объем'])
            list_of_constraints[constraint] = (cvxopt.modeling.sum(trans) >= const)
            
            print("mass{} = (".format(number_of_constraints) + string + ") >= " + str(const) + ")")
            print("")
            
            count = count + 1
            number_of_constraints += 1
            number_of_x = number_of_x + values_n_points_i

mass181 = (x[0] + x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + x[17] + x[18]) <= 331.1)

mass182 = (x[0] + x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + x[17] + x[18]) >= 165.55)

mass183 = (x[19] + x[20] + x[21] + x[22] + x[23] + x[24] + x[25] + x[26] + x[27] + x[28] + x[29] + x[30] + x[31] + x[32] + x[33] + x[34] + x[35] + x[36] + x[37]) <= 1000000.0)

mass184 = (x[19] + x[20] + x[21] + x[22] + x[23] + x[24] + x[25] + x[26] + x[27] + x[28] + x[29] + x[30] + x[31] + x[32] + x[33] + x[34] + x[35] + x[36] + x[37]) >= 165.55)

mass185 = (x[38] + x[39] + x[40] + x[41] + x[42] + x[43] + x[44] + x[45] + x[46] + x[47] + x[48] + x[49] + x[50] + x[51] + x[52] + x[53] + x[54] + x[55] + x[56] + x[57] + x[58] + x[59] + x[60] + x[61] + x[62] + x[63] + x[64] + x[65] + x[66] + x[67] + x[68] + x[69] + x[70] + x[71] + x[72] + x[73] + x[74] + x[75] + x[76] +

In [27]:
#неотрицательные объемы
x_non_negative = (x >= 0)   

In [28]:
list_of_constraints['x_non_negative'] = x_non_negative

In [29]:
len(list_of_constraints)

201

In [30]:
# Final constraints
c = pd.DataFrame(list_of_constraints.items())
c = list(c[:][1].values)
c

[<scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,
 <scalar equality>,


### Solving the problem

In [31]:
start = time.time()


problem =op(z, c)

problem.solve(solver='glpk')  

print("Результат Xopt:")
for i in x.value:
    print ("x =", i)

print("Стоимость доставки:")
print(problem.objective.value()[0])

stop = time.time()
print ("Время :")
print(stop - start)

GLPK Simplex Optimizer 5.0
560 rows, 360 columns, 1440 non-zeros
      0: obj =   0.000000000e+00 inf =   3.699e+04 (190)
    190: obj =   9.804661523e+07 inf =   0.000e+00 (0)
*   372: obj =   8.558364440e+07 inf =   0.000e+00 (0) 1
OPTIMAL LP SOLUTION FOUND
Результат Xopt:
x = 0.0
x = 0.0
x = 69.61739907583812
x = 7.105427357601002e-15
x = 33.14470001081621
x = 0.0
x = 42.38466312255857
x = 18.71718475620516
x = 0.06038304422341767
x = 0.0
x = 0.0
x = 0.0
x = 4.73793339941212
x = 15.23688750856649
x = 7.962316122394742
x = 14.27967784390616
x = 8.315718258401118
x = 10.24538436128756
x = 0.0
x = 29.69380349634518
x = 23.71778272032937
x = 0.0
x = 17.180032468350984
x = 0.0
x = 19.22090952214534
x = 0.0
x = 0.0
x = 29.892012621136423
x = 8.060744781665605
x = 13.67892325874485
x = 8.212596603459344
x = 0.0
x = 0.0
x = 0.0
x = 0.0
x = 0.0
x = 0.0
x = 15.89319452782291
x = 5.684341886080802e-14
x = 240.3026994876928
x = 193.4257680930644
x = 156.7413584483274
x = 199.0286364293773
x = 0

### Postprocessing

In [32]:
# Defining the final products' volumes
def values_costs_for_brand(matrixes):
    volumes_result = {}
    i = 0
    n_brands = data['НГ'].nunique()
    n_origins = max(data['НБ'].str.strip('Нефтебаза ').astype('int'))
    n_points = max(data['ОУ'].str.strip('АЗС ').astype('int'))
    volumes = list(x.value)
    for b in np.arange(0, n_brands):
        matrix_b = matrixes['matrix_brand_{b}'.format(b=b)]
        for o in np.arange(0, n_origins+1):
            for p in np.arange(0, n_points+1):

                if (o in matrix_b.index) and (p in matrix_b.columns):
                    name_of_product = "P_" + str(p) + "_O_"+ str(o) + "_brand_"+ str(b)
                    volumes_result[name_of_product] = volumes[i]
                    i += 1 
    volumes_result_data = pd.DataFrame(volumes_result.items(), columns=['Product', 'Value'])
    return volumes_result_data

volumes_result_data = values_costs_for_brand(matrixes)
volumes_result_data

,Product,Value
0,P_0_O_0_brand_0,0.000000e+00
1,P_1_O_0_brand_0,0.000000e+00
2,P_3_O_0_brand_0,6.961740e+01
3,P_4_O_0_brand_0,7.105427e-15
4,P_7_O_0_brand_0,3.314470e+01
...,...,...
355,P_44_O_1_brand_4,0.000000e+00
356,P_45_O_1_brand_4,0.000000e+00
357,P_46_O_1_brand_4,8.804172e+01
358,P_47_O_1_brand_4,0.000000e+00
